In [1]:
import pandas as pd
import glob
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, cross_validate
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder

In [2]:
# Grayson's path
train_folder = "C:\\Users\\grays\\Will-Grayson GitHub Repo\\will-grayson-ML\\six-class\\data\\train\\"
test_folder = "C:\\Users\\grays\\Will-Grayson GitHub Repo\\will-grayson-ML\\six-class\\data\\test\\"

# Will's path (comment out if not in use)
# train_folder = "C:\\Users\\willg\\OneDrive\\CSCI\\summer-2024-work\\will-grayson-ML\\train\\"
# test_folder = "C:\\Users\\willg\\OneDrive\\CSCI\\summer-2024-work\\will-grayson-ML\\test\\"

# Function to load and concatenate CSV files from a list of file paths
def load_and_concat(files, sample_fraction=None):
    df_list = []
    for file in files:
        df = pd.read_csv(file)
        if sample_fraction:
            df = df.sample(frac=sample_fraction, random_state=42)  # Random sampling
        df_list.append(df)
    return pd.concat(df_list, ignore_index=True)

# List all CSV files in the train and test folders
train_files = glob.glob(train_folder + "*.csv")
test_files = glob.glob(test_folder + "*.csv")

# Load and concatenate training and testing data
train_df = load_and_concat(train_files, sample_fraction=0.1)  # Use 10% of the data
test_df = load_and_concat(test_files, sample_fraction=0.1)

In [3]:
# Ensure the columns match by selecting the same feature columns in both train and test sets
feature_columns = [col for col in train_df.columns if col not in ['spoofing', 'benign', 'MQTT', 'recon', 'DDoS', 'DoS']]

# Create a new target column for the 6-class classification
def determine_class(row):
    if row['spoofing'] == 1:
        return 'spoofing'
    elif row['benign'] == 1:
        return 'benign'
    elif row['MQTT'] == 1:
        return 'MQTT'
    elif row['recon'] == 1:
        return 'recon'
    elif row['DDoS'] == 1:
        return 'DDoS'
    elif row['DoS'] == 1:
        return 'DoS'
    else:
        return 'unknown'

train_df['Attack_Type'] = train_df.apply(determine_class, axis=1)
test_df['Attack_Type'] = test_df.apply(determine_class, axis=1)

# Separate features and labels
X_train = train_df[feature_columns]
y_train = train_df['Attack_Type']
X_test = test_df[feature_columns]
y_test = test_df['Attack_Type']


In [4]:
# Encode the labels to numerical values
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.transform(y_test)

# Create a scaler object
scaler = MinMaxScaler()

# Fit the scaler to the training features and transform both training and testing features
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert the scaled features to DataFrames
X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=feature_columns)
X_test_scaled_df = pd.DataFrame(X_test_scaled, columns=feature_columns)

In [5]:
# Initialize the Random Forest Classifier
model = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the model
model.fit(X_train_scaled, y_train)

# Make predictions
y_pred = model.predict(X_test_scaled)

# Evaluate the model
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[110971    740      0      0      0      0]
 [    22  42809      0      0      0      0]
 [     0      0    125     33      1     16]
 [     0      0      0   3688     14     59]
 [     0      0      0     90   2665     12]
 [     0      0      0     24     21    129]]
              precision    recall  f1-score   support

           0       1.00      0.99      1.00    111711
           1       0.98      1.00      0.99     42831
           2       1.00      0.71      0.83       175
           3       0.96      0.98      0.97      3761
           4       0.99      0.96      0.97      2767
           5       0.60      0.74      0.66       174

    accuracy                           0.99    161419
   macro avg       0.92      0.90      0.90    161419
weighted avg       0.99      0.99      0.99    161419

